In [ ]:
!nvidia-smi

In [1]:
from keras import models
from PIL import Image, ImageOps
import numpy as np
import keras
from keras import layers
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import threading as th
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import pickle as pick
from numba import njit

In [2]:
def resampleImage(fpath):
  image = Image.open(fpath).convert("RGB")
  size = (244, 244)
  image = ImageOps.fit(image, size)
  image_array = np.asarray(image)
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
  return normalized_image_array

def random_invert_img(x, p=0.5):
  if tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

def random_invert(factor=0.5):
  return keras.layers.Lambda(lambda x: random_invert_img(x, factor))

def getDirFiles(fpath):
  onlyfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
  return onlyfiles

class RandomInvert(keras.layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Take photo';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

        // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# try:
# if True:
#   model = models.load_model("./drive/MyDrive/my_model.h5")
# model.compile(keras.optimizers.nadam_experimental.Nadam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.Accuracy(), keras.metrics.Precision(), keras.metrics.Recall(), tf.keras.metrics.AUC()])

# Ниже - идеальная модель

# except:
if True:
  model = keras.Sequential()
  model.add(layers.InputLayer((244, 244, 3)))
  # model.add(RandomInvert(0.5)) # Пока что не нужно, мешает

  model.add(layers.Conv2D(32, 5, activation="relu"))
  model.add(layers.MaxPooling2D())

  model.add(layers.Conv2D(64, 5, activation="relu"))
  model.add(layers.MaxPooling2D())

  model.add(layers.Conv2D(32, 3, activation="relu"))
  model.add(layers.MaxPooling2D())

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1, activation="softmax"))

  model.summary()
  model.compile(keras.optimizers.nadam_experimental.Nadam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.Accuracy(), keras.metrics.Precision(), keras.metrics.Recall()])


In [1]:
# Clear cache
tf.keras.backend.clear_session()

normal_n = getDirFiles("./drive/MyDrive/sorted/normal")

myopia = getDirFiles("./drive/MyDrive/sorted/myopia")
cataract = getDirFiles("./drive/MyDrive/sorted/cataract")
atrophy = getDirFiles("./drive/MyDrive/sorted/atrophy")
degeneration = getDirFiles("./drive/MyDrive/sorted/degeneration")
diabetic = getDirFiles("./drive/MyDrive/sorted/diabetic")
drusen = getDirFiles("./drive/MyDrive/sorted/drusen")
glaucoma = getDirFiles("./drive/MyDrive/sorted/glaucoma")
hypertensive = getDirFiles("./drive/MyDrive/sorted/hypertensive")
macular = getDirFiles("./drive/MyDrive/sorted/macular")
retinopathy = getDirFiles("./drive/MyDrive/sorted/retinopathy")

ill = []
normal = []

for i in myopia:
  ill.append("./drive/MyDrive/sorted/myopia/"+i)

for i in cataract:
  ill.append("./drive/MyDrive/sorted/cataract/"+i)

for i in atrophy:
  ill.append("./drive/MyDrive/sorted/atrophy/"+i)

for i in degeneration:
  ill.append("./drive/MyDrive/sorted/degeneration/"+i)

for i in diabetic:
  ill.append("./drive/MyDrive/sorted/diabetic/"+i)

for i in drusen:
  ill.append("./drive/MyDrive/sorted/drusen/"+i)
  
for i in glaucoma:
  ill.append("./drive/MyDrive/sorted/glaucoma/"+i)

for i in hypertensive:
  ill.append("./drive/MyDrive/sorted/hypertensive/"+i)

for i in macular:
  ill.append("./drive/MyDrive/sorted/macular/"+i)

for i in retinopathy:
  ill.append("./drive/MyDrive/sorted/retinopathy/"+i)

for i in normal_n:
  normal.append("./drive/MyDrive/sorted/normal/"+i)

del myopia, cataract, atrophy, degeneration, diabetic, drusen, glaucoma, hypertensive, macular, retinopathy, normal_n

x_data = np.ndarray(shape=(len(normal)+len(ill)+1, 244, 244, 3), dtype=np.float32)
y_data = np.ndarray(shape=(len(normal)+len(ill)+1, 244, 244, 3), dtype=np.float32)

b = 0

i = 0

while i <= len(ill) or i <= len(normal):
  if i < len(normal):
    resImg1 = resampleImage(normal[i])
    x_data[b] = resImg1
    y_data[b] = np.asarray([1])
    b+=1
  if i < len(ill):
    resImg2 = resampleImage(ill[i])
    x_data[b+1] = resImg2
    y_data[b+1] = np.asarray([0])
    b+=1

  i+=1

del normal, ill, i, b

NameError: ignored

In [ ]:
with open("./drive/MyDrive/data.pk1", "wb", -1) as f:
  pick.dump({"y_data": y_data, "x_data": x_data}, f)

del y_data, x_data

In [ ]:
with open("./drive/MyDrive/data.pk1", "rb", -1) as f:
  data = pick.load(f)

  x_data = data["x_data"]
  y_data = data["y_data"]
model.fit(np.asarray(x_data), np.asarray(y_data), batch_size=32, epochs=200)
model.save("./drive/MyDrive/my_model.h5")

del x_data, y_data

In [6]:
labels = ["Ваши глаза здоровы!", "Возможно, что ваши глаза больны."]
# Camera
try:
  take_photo("photo.jpg", 0.3)
  
  image = Image.open("./photo.jpg").convert("RGB")
  size = (244, 244)
  image = ImageOps.fit(image, size)
  image_array = np.asarray(image).reshape((244, 244, 3))

  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
  m_data = np.ndarray(shape=(1, 244, 244, 3), dtype=np.float32)

  m_data[0] = normalized_image_array
  res = model.predict(m_data)

  diagnose = labels[np.argmax(res)]
  print("Результат: {}".format(diagnose))

except Exception as err:
  print(str(err))

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 211ms/step
Результат: Ваши глаза здоровы!
